In [1]:
!pip3 install tensorflow_text>=2.0.0rc0


In [21]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


In [22]:
data=pd.read_csv('/content/data_with_predictions.csv')

In [23]:

data=data.dropna()
data=data.drop('commentaire',axis=1)

In [24]:
max_words = 10000  # Nombre maximum de mots à prendre en compte
max_len = 100  # Longueur maximale d'une séquence

# Tokenisation des articles
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['predicted_problems'])
sequences = tokenizer.texts_to_sequences(data['predicted_problems'])

In [25]:
labels = pd.get_dummies(data['predicted_problems']).values

In [26]:
# Padding des séquences
padded_sequences = pad_sequences(sequences, maxlen=max_len)

# Conversion des catégories en labels numériques
labels = pd.get_dummies(data['predicted_problems']).values
train_ratio = 0.8  # Pourcentage d'articles à utiliser pour l'entraînement
train_size = int(len(padded_sequences) * train_ratio)
train_data = padded_sequences[:train_size]
train_labels = labels[:train_size]
test_data = padded_sequences[train_size:]
test_labels = labels[train_size:]

In [27]:
embedding_dim = 100  # Dimension de l'espace d'incorporation des mots

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len))
model.add(LSTM(128))
model.add(Dense(labels.shape[1], activation='softmax'))


In [28]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
# Entraînement du modèle
epochs = 100  # Nombre d'époques d'entraînement
batch_size = 32  # Taille du lot
model.fit(train_data, train_labels, epochs=epochs, batch_size=batch_size, validation_split=0.2)

# Évaluation du modèle sur les données de test
loss, accuracy = model.evaluate(test_data, test_labels, batch_size=batch_size)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Epoch 1/100
213/213 [==============================] - 7s 19ms/step - loss: 1.3439 - accuracy: 0.7144 - val_loss: 2.2610 - val_accuracy: 0.5618
Epoch 2/100
213/213 [==============================] - 2s 8ms/step - loss: 0.3697 - accuracy: 0.9247 - val_loss: 1.4325 - val_accuracy: 0.7118
Epoch 3/100
213/213 [==============================] - 2s 8ms/step - loss: 0.1787 - accuracy: 0.9684 - val_loss: 1.0473 - val_accuracy: 0.8171
Epoch 4/100
213/213 [==============================] - 2s 10ms/step - loss: 0.1008 - accuracy: 0.9826 - val_loss: 0.8605 - val_accuracy: 0.8418
Epoch 5/100
213/213 [==============================] - 2s 10ms/step - loss: 0.0641 - accuracy: 0.9898 - val_loss: 0.7581 - val_accuracy: 0.8800
Epoch 6/100
213/213 [==============================] - 2s 8ms/step - loss: 0.0425 - accuracy: 0.9931 - val_loss: 0.6893 - val_accuracy: 0.9065
Epoch 7/100
213/213 [==============================] - 2s 8ms/step - loss: 0.0297 - accuracy: 0.9957 - val_loss: 0.6358 - val_accuracy: 0.9

In [39]:
# Supposons que vous avez une nouvelle phrase en arabe à classifier
new_text=  "Mauvais equipement  qualité des produits 0 le service 0 les prix très élevés,Trop de bruit .."
# Prétraitement de la nouvelle phrase
new_sequence = tokenizer.texts_to_sequences([new_text])
new_padded_sequence = pad_sequences(new_sequence, maxlen=max_len)

# Faire la prédiction
predictions = model.predict(new_padded_sequence)

# Convertir les probabilités en catégories
predicted_category = np.argmax(predictions, axis=1)

# Récupérer le nom de la catégorie prédite
category_labels = data['predicted_problems'].unique()
predicted_category_label = category_labels[predicted_category[0]]

print("Phrase :", new_text)
print("Catégorie prédite :", predicted_category_label)


1/1 [==============================] - 0s 18ms/step
Phrase : Mauvais equipement  qualité des produits 0 le service 0 les prix très élevés,Trop de bruit ..
Catégorie prédite : ['Qualité du Service', "Problèmes d'Équipements et d'Installations", 'Bruits', 'Problèmes de Prix']
